<a href="https://colab.research.google.com/github/parwisenlared/MasterThesis/blob/master/GAKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code from GAKeras repository, trying to see if it works.

In [1]:
pip install deap

In [2]:
# Algorithm

import random
import pickle
import time
import datetime 

from deap import algorithms 


def myEASimple(population, start_gen, toolbox, cxpb, mutpb, ngen, 
               stats, halloffame, logbook, verbose, id=None):

    total_time = datetime.timedelta(seconds=0) 
    for gen in range(start_gen, ngen):
        start_time = datetime.datetime.now()
        population = algorithms.varAnd(population, toolbox, cxpb=cxpb, mutpb=mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in population if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        halloffame.update(population)
        record = stats.compile(population)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)


        population = toolbox.select(population, k=len(population))

        if gen % 1 == 0:
            # Fill the dictionary using the dict(key=value[, ...]) constructor
            cp = dict(population=population, generation=gen, halloffame=halloffame,
                      logbook=logbook, rndstate=random.getstate())
            if id is None:
                cp_name = "checkpoint_ea.pkl"
            else:
                cp_name = "checkpoint_ea_{}.pkl".format(id)
            pickle.dump(cp, open(cp_name, "wb"))
            
        gen_time = datetime.datetime.now() - start_time
        total_time = total_time + gen_time
        #print("Time ", total_time)
        if total_time > datetime.timedelta(hours=4*24):
            print("Time limit exceeded.")
            break 

    return population, logbook


def myEAMuCommaLambda(population, startgen, toolbox, mu, lambda_, cxpb, mutpb, ngen,
                      stats=None, halloffame=None, logbook=None, verbose=False, id=None):

    assert lambda_ >= mu, "lambda must be greater or equal to mu."

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    if logbook is None:
        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=startgen, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    total_time = datetime.timedelta(seconds=0) 
    for gen in range(startgen+1, ngen):
        start_time = datetime.datetime.now()
        
        # Vary the population
        offspring = algorithms.varOr(population, toolbox, lambda_, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Select the next generation population
        population[:] = toolbox.select(offspring, mu)

        # Update the statistics with the new population
        record = stats.compile(population) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

        if gen % 1 == 0:
            # Fill the dictionary using the dict(key=value[, ...]) constructor
            cp = dict(population=population, generation=gen, halloffame=halloffame,
                      logbook=logbook, rndstate=random.getstate())
            if id is None:
                cp_name = "checkpoint_es.pkl"
            else:
                cp_name = "checkpoint_es_{}.pkl".format(id)
            pickle.dump(cp, open(cp_name, "wb"))

        gen_time = datetime.datetime.now() - start_time
        total_time = total_time + gen_time
        if total_time > datetime.timedelta(hours=4*24):
            print("Time limit exceeded.")
            break 

            
            
    return population, logbook

In [3]:
# Configuration MNIST

class Cfg:
    pass 

class CfgSensors:

    batch_size = 100
    epochs = 500
    loss = 'mean_squared_error'

    task_type = 'approximation'

    pop_size = 10
    ngen = 30

    MAX_LAYERS = 5
    MAX_LAYER_SIZE = 100
    MIN_LAYER_SIZE = 5
    DROPOUT = [ 0.0, 0.2, 0.3, 0.4 ] 
    ACTIVATIONS = [ 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear' ] 


class CfgSensorsES:

    batch_size = 100
    epochs = 500
    loss = 'mean_squared_error'

    task_type = 'approximation'

    MU = 10
    LAMBDA = 30
    ngen = 100

    MAX_LAYERS = 5
    MAX_LAYER_SIZE = 100
    MIN_LAYER_SIZE = 5
    DROPOUT = [ 0.0, 0.2, 0.3, 0.4 ] 
    ACTIVATIONS = [ 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear' ] 

    SIZE_MIN_STRATEGY = 5
    SIZE_MAX_STRATEGY = 50
    DROPOUT_MIN_STRATEGY = 0.05
    DROPOUT_MAX_STRATEGY = 0.2

    
    
class CfgMnist:

    batch_size = 128
    epochs = 10
    loss = 'categorical_crossentropy'
    #loss = 'mean_squared_error'
    
    task_type = "classification"

    pop_size = 20
    ngen = 300

    MAX_LAYERS = 5
    MAX_LAYER_SIZE = 300
    MIN_LAYER_SIZE = 10 
    DROPOUT = [ 0.0, 0.2, 0.3, 0.4 ] 
    ACTIVATIONS = [ 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear' ] 

    # for convolutional networks
    MIN_FILTERS = 10
    MAX_FILTERS = 50 
    MIN_KERNEL_SIZE = 2
    MAX_KERNEL_SIZE = 5
    MIN_POOL_SIZE = 2
    MAX_POOL_SIZE = 3
    MAX_CONV_LAYERS = 3
    MAX_DENSE_LAYERS = 3
    
    #DENSE_LAYER = 0.5
    CONV_LAYER = 0.7 
    MAX_POOL_LAYER = 0.3 

    
    
class CfgMnistES:

    batch_size = 128
    epochs = 20 
    #loss = 'categorical_crossentropy'
    loss = 'mean_squared_error'
    
    task_type = "classification"

    MU = 5
    LAMBDA = 10
    ngen = 10

    SIZE_MIN_STRATEGY = 5
    SIZE_MAX_STRATEGY = 50
    DROPOUT_MIN_STRATEGY = 0.05
    DROPOUT_MAX_STRATEGY = 0.2
    
    MAX_LAYERS = 5
    MAX_LAYER_SIZE = 1000
    MIN_LAYER_SIZE = 10 
    DROPOUT = [ 0.0, 0.2, 0.3, 0.4 ] 
    ACTIVATIONS = [ 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear' ] 

#Config = CfgSensors()    
Config = CfgMnist()
#Config = CfgSensorsES()  
#Config = CfgMnist()

import configparser
import re

def is_int(s):
    if re.fullmatch(r'[0-9]+', s):
        return True
    else:
        return False

def is_float(s):
    if re.fullmatch(r'[0-9]+\.[0-9]+', s):
        return True
    else:
        return False

def is_list(s):
    if re.fullmatch(r'\[.+\]', s):
        return True
    else:
        return False

def convert(s):
    if is_int(s):
        val = int(s)
    elif is_float(s):
        val = float(s)
    elif is_list(s):
        s = s.strip()
        s = s.strip("[")
        s = s.strip("]")
        val = s.split(',')
        newval = [] 
        for v in val:
            v = v.strip()
            v = v.strip("'")
            v = v.strip('"')
            newval.append(convert(v))
        val = newval
    else:
        val = s
    return val
    
    
def load_config(name):

    config = configparser.ConfigParser()
    config.read(name)
    
    global Config
    Config = Cfg()
    
    for sec in config.sections():
        for key, val in config[sec].items():
            val = convert(val)
            setattr(Config, key.lower(), val)
            setattr(Config, key.upper(), val)

In [4]:
# Utils

import random 


def roulette(functions, probs):
    r = random.random()
    for func, prob in zip(functions, probs):
        if r < prob:
            return func
        else:
            r -= prob
    return None
                
import numpy as np

def mean_sq_error(y1, y2):
    diff = y1 - y2
    E = 100 * sum(diff*diff) / len(y1)
    return E

def accuracy_score(y1, y2):
    assert y1.shape == y2.shape
    
    y1_argmax = np.argmax(y1, axis=1)
    y2_argmax = np.argmax(y2, axis=1)
    score = sum(y1_argmax == y2_argmax)
    return score / len(y1)
        
    
def error(y1, y2):
    if Config.task_type == "classification":
        return accuracy_score(y1, y2)
    else:
        return mean_sq_error(y1, y2)


In [5]:
# Individual 

import random
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop

#from config import Config


class Layer:
    """ Specification of one layer.
        Includes size, regularization, activaton. 
    """
    def __init__(self):
        pass

    def randomInit(self):
        self.size = random.randint(Config.MIN_LAYER_SIZE, Config.MAX_LAYER_SIZE)
        self.dropout = random.choice(Config.DROPOUT) 
        self.activation = random.choice(Config.ACTIVATIONS)  
        return self

    def __str__(self):
        return " #{} dropout={} activation={}".format(self.size, self.dropout, self.activation)


class Individual:
    """  Individual coding network architecture. 
    """
    
    def __init__(self):
        self.input_shape = Config.input_shape
        self.noutputs = Config.noutputs 
        #print(self.input_shape, self.noutputs)

        
    def randomInit(self):
        self.layers = []
        num_layers = random.randint(1, Config.MAX_LAYERS)
        for l in range(num_layers):
            layer = Layer().randomInit() 
            self.layers.append(layer)

    def createNetwork(self):

        model = Sequential()

        firstlayer = True
        for l in self.layers:
            if firstlayer:
                model.add(Dense(l.size, input_shape=self.input_shape))
                firstlayer = False
            else:
                model.add(Dense(l.size))
            model.add(Activation(l.activation))
            if l.dropout > 0:
                model.add(Dropout(l.dropout))

        # final part 
        model.add(Dense(self.noutputs))
        if Config.task_type == "classification":
            model.add(Activation('softmax'))
            
        model.compile(loss=Config.loss,
                      optimizer=RMSprop())
        
        return model 

    def __str__(self):

        ret = "------------------------\n"
        for l in self.layers:
            ret += str(l)
            ret += "\n" 
            ret += "------------------------\n"
        return ret 

def initIndividual(indclass):
    ind = indclass()
    ind.randomInit()
    return ind

Using TensorFlow backend.


In [6]:
# Configuration individual

import random
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
#from individual import Layer

class ConvLayer:
    """ Specification of one convolutional layer.
        Includes number of filters, kernel size, activation.
    """
    def __init__(self):
        pass

    def randomInit(self):
        self.filters = random.randint(Config.MIN_FILTERS, Config.MAX_FILTERS)
        # filters are squares kernel_size x kernel_size 
        self.kernel_size = random.randint(Config.MIN_KERNEL_SIZE, Config.MAX_KERNEL_SIZE)
        self.activation = random.choice(Config.ACTIVATIONS)
        return self
    
    def __str__(self):
        return "conv #{} kernelsize={} activation={}".format(self.filters, self.kernel_size, self.activation)
        

class MaxPoolLayer:
    """ Specification of one max pooling layer.
    """
    def __init__(self):
        pass
    
    def randomInit(self):
        # pooling size is (pool_size, pool_size)
        self.pool_size =  random.randint(Config.MIN_POOL_SIZE, Config.MAX_POOL_SIZE)
        return self

    def __str__(self):
        return "pool poolsize={} ".format(self.pool_size)


def createRandomLayer():

    create = roulette([lambda: ConvLayer(),
                       lambda: MaxPoolLayer()],
                      [Config.CONV_LAYER, Config.MAX_POOL_LAYER])
    if create:
        return create()
    else:
        return ConvLayer() 
    
class ConvIndividual:
    """ Individual coding convolutional network architecture.
        Individual consists of two parts, first of convolutioanal
        and max pooling layers, the second part of dense layers.
    """

    def __init__(self):
        self.input_shape = Config.input_shape
        self.noutputs = Config.noutputs
        self.nparams = None
        
    def randomInit(self):
        self.conv_layers = []
        num_conv_layers = random.randint(1, Config.MAX_CONV_LAYERS)
        for l in range(num_conv_layers):
            layer = createRandomLayer().randomInit()
            self.conv_layers.append(layer)
            
        self.dense_layers = []
        num_dense_layers = random.randint(1, Config.MAX_DENSE_LAYERS)
        for l in range(num_dense_layers):
            layer = Layer().randomInit()
            self.dense_layers.append(layer)


    def createNetwork(self):

        model = Sequential()


        firstlayer = True

        # convolutional part 
        for l in self.conv_layers:
            if type(l) is ConvLayer:
                if firstlayer:
                    model.add(Conv2D(l.filters, (l.kernel_size, l.kernel_size),
                                     padding='same', # let not the shape vanish
                                     input_shape=self.input_shape))
                    firstlayer = False
                else:
                    model.add(Conv2D(l.filters, (l.kernel_size, l.kernel_size), padding='same'))
                model.add(Activation(l.activation))
                
            elif type(l) is MaxPoolLayer:
                if firstlayer:
                    model.add(MaxPooling2D(pool_size=(l.pool_size,l.pool_size),
                                           input_shape=self.input_shape))
                    firstlayer = False
                else:
                    # check if pooling is possible
                    if model.layers[-1].output_shape[1] >= l.pool_size and model.layers[-1].output_shape[2] >= l.pool_size:
                        model.add(MaxPooling2D(pool_size=(l.pool_size, l.pool_size)))
                    
            else:
                raise TypeError("unknown type of layer") 
            
        # dense part
        model.add(Flatten())
        for l in self.dense_layers:
            model.add(Dense(l.size))
            model.add(Activation(l.activation))
            if l.dropout > 0:
                model.add(Dropout(l.dropout))

        # final part 
        model.add(Dense(self.noutputs))
        if Config.task_type == "classification":
            model.add(Activation('softmax'))
        
        model.compile(loss=Config.loss,
                      optimizer=RMSprop())

        self.nparams = model.count_params()
                
        return model 

    
    def __str__(self):

        ret = "------------------------\n"
        for l in self.conv_layers+self.dense_layers:
            ret += str(l)
            ret += "\n" 
            ret += "------------------------\n"
        return ret 

In [7]:
# Configuration configuration

import configparser

config = configparser.ConfigParser()

config['DEFAULT'] = { 'batch_size': 128,
                      'epochs': 20,
                      'loss': 'mean_squared_error',
                      'task_type': 'classification' }

config['ES'] = { 'mu': 5,
                 'lambda': 10,
                 'ngen': 20 }

config['STRATEGY_COEFS'] = { 'size_min_strategy': 5,
                             'size_max_strategy': 50,
                             'dropout_min_strategy': 0.05,
                             'dropout_max_strategy': 0.2 }

config['NETWORK'] = {'max_layers': 5,
                     'max_layer_size': 1000,
                     'min_layer_size': 10,
                     'dropout': [ 0.0, 0.2, 0.3, 0.4 ],
                     'activations': [ 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear' ] }

with open('example.ini', 'w') as configfile:
    config.write(configfile)


In [8]:
# Crossover

import random 

def cxListOnePoint(list1, list2):
    assert len(list1) > 0 and len(list2) > 0
    
    cxpoint1 = random.randint(0, len(list1) - 1)
    cxpoint2 = random.randint(0, len(list2) - 1)
    list1[cxpoint1:], list2[cxpoint2:] = list2[cxpoint2:], list1[cxpoint1:]
    
    assert len(list1) > 0 and len(list2) > 0 
    

class Crossover:

    def cxOnePoint(self, ind1, ind2):
        #do one-point crossover on list of layers 
        cxListOnePoint(ind1.layers, ind2.layers) 
        return ind1, ind2 

    
class CrossoverConv:

    def cxOnePoint(self, ind1, ind2):
        #do one-point crossover on list of layers
        #convolutional layers
        cxListOnePoint(ind1.conv_layers, ind2.conv_layers)
        #dense layers 
        cxListOnePoint(ind1.dense_layers, ind2.dense_layers)
        
        return ind1, ind2 


In [9]:
# Crossover ES

import random 

class CrossoverES:

    def cxOnePoint(self, ind1, ind2):
        #do one-point crossover on list of layers 
        cxpoint1 = random.randint(0, len(ind1.layers) - 1)
        cxpoint2 = random.randint(0, len(ind2.layers) - 1)

        ind1.layers[cxpoint1:], ind2.layers[cxpoint2:] = ind2.layers[cxpoint2:], ind1.layers[cxpoint1:]

        s1, s2  = ind1.strategy, ind2.strategy
        s1.blocks[cxpoint1:], s2.blocks[cxpoint2:] = s2.blocks[cxpoint2:], s1.blocks[cxpoint1:] 
        
        assert len(ind1.layers) > 0
        assert len(ind2.layers) > 0 
        assert len(ind1.layers) == len(ind1.strategy.blocks)
        assert len(ind2.layers) == len(ind2.strategy.blocks)
        
        return ind1, ind2 

In [10]:
# Dataset 
import sys
from matplotlib import pyplot
from keras.datasets import cifar10, mnist
from keras.utils import to_categorical

def load_dataset():
	# load dataset
  (trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
  X_train = trainX.reshape((trainX.shape[0], 28, 28, 1))
  X_test = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
  Y_train = to_categorical(trainY)
  Y_test = to_categorical(testY)
  print("Data loaded")
  print("Preparing pixels")
  # convert from integers to floats
  train_norm = X_train.astype('float32')
  test_norm = X_test.astype('float32')
  # normalize to range 0-1
  X_train = train_norm / 255.0
  X_test = test_norm / 255.0
  # return normalized images
  print("Pixel data ready")

  return X_train, Y_train, X_test, Y_test

In [11]:
# Fitness 

import random 
import numpy as np 
import pickle
import sklearn
from sklearn.model_selection import KFold
from keras import backend as K 


class Database:

    def __init__(self):
        self.data = []
    
    def insert(self, individual, fitness):
        self.data.append((individual, fitness))
        
    def save(self, name):
        with open(name, "wb") as f:
            pickle.dump(self.data, f)

class Fitness:

    def __init__(self, xtrain, ytrain):
        
        # load train data 
        self.X = xtrain
        self.y = ytrain
                
    def evaluate(self, individual):
        #print(" *** evaluate *** ")

        #model = individual.createNetwork()
        #return random.random(), 
         
        random.seed(42) 
        # perform KFold crossvalidation 
        kf = KFold(n_splits=5)
        scores = []
        for train, test in kf.split(self.X):   # train, test are indicies 
            X_train, X_test = self.X[train], self.X[test]
            y_train, y_test = self.y[train], self.y[test]
                
            model = individual.createNetwork()
            model.fit(X_train, y_train,
                      batch_size=Config.batch_size, epochs=Config.epochs, verbose=0)
            
            yy_test = model.predict(X_test)
            scores.append(error(y_test, yy_test))
            
        fitness = np.mean(scores)
            
        # I try this to prevent memory leaks in nsga2-keras 
        K.clear_session()

        return fitness

In [12]:
# Mutation for NETWORK

import random

PROB_MUTATE_LAYER = 0.5
PROB_ADD_LAYER = 0.25
PROB_DEL_LAYER = 0.25

PROB_CHANGE_LAYER_SIZE = 0.4
PROB_CHANGE_DROPOUT = 0.25
PROB_CHANGE_ACTIVATION = 0.25
PROB_CHANGE_ALL = 0.1 

PROB_RANDOM_LAYER_SIZE = 0.4
PROB_ADD_NEURON = 0.3
PROB_DEL_NEURON = 0.3 

class Mutation:

    def __init__(self):
        pass


    def changeSize(self, layer):
        layer.size = random.randint(Config.MIN_LAYER_SIZE, Config.MAX_LAYER_SIZE)

    def addNeuron(self, layer):
        layer.size += 1

    def delNeuron(self, layer):
        if layer.size > 0:
            layer.size -= 1 
    
    def mutateSize(self, layer):

        mutfunc = roulette([self.changeSize, self.addNeuron, self.delNeuron],
                           [PROB_RANDOM_LAYER_SIZE, PROB_ADD_NEURON, PROB_DEL_NEURON])
        if mutfunc:
            mutfunc(layer)


    def mutateDropout(self, layer):
        layer.dropout = random.choice(Config.DROPOUT)
        
    def mutateActivation(self, layer):
        layer.activation = random.choice(Config.ACTIVATIONS)

    def randomInit(self, layer):
        layer.randomInit()

    def mutateLayer(self, individual):

        # select layer random 
        l = random.randint(0, len(individual.layers)-1)
        layer = individual.layers[l]

        mutfunc = roulette([self.mutateSize, self.mutateDropout, self.mutateActivation,
                            self.randomInit],
                           [PROB_CHANGE_LAYER_SIZE, PROB_CHANGE_DROPOUT, PROB_CHANGE_ACTIVATION,
                            PROB_CHANGE_ALL])

        if mutfunc:
            mutfunc(layer)
        
        return individual,

    def addLayer(self, individual):
        l = random.randint(0, len(individual.layers)-1)
        individual.layers.insert(l, Layer().randomInit())
        return individual, 

    def delLayer(self, individual):
        if len(individual.layers)>1:
            l = random.randint(0, len(individual.layers)-1)
            del individual.layers[l]
        return individual, 
    
    def mutate(self, individual): 
        
        mutfunc = roulette([self.mutateLayer, self.addLayer, self.delLayer],
                           [PROB_MUTATE_LAYER, PROB_ADD_LAYER, PROB_DEL_LAYER])
        if mutfunc:
            return mutfunc(individual)
        
        return individual, 




# Mutation in COnv Layers
PROB_CHANGE_NUM_FILTERS = 0.3
PROB_MUTATE_KERNEL_SIZE = 0.3
PROB_CHANGE_CONV_ACTIVATION = 0.3
PROB_CHANGE_CONV_ALL = 0.1



class MutationConv(Mutation):

    def __init__(self):
        pass


    def mutateFilters(self, layer):
        layer.filters = random.randint(Config.MIN_FILTERS, Config.MAX_FILTERS)

    def mutateKernelSize(self, layer):
        layer.kernel_size = random.randint(Config.MIN_KERNEL_SIZE, Config.MAX_KERNEL_SIZE)

    def mutatePoolSize(self, layer):
        layer.pool_size =  random.randint(Config.MIN_POOL_SIZE, Config.MAX_POOL_SIZE)
        
        
    def mutateLayer(self, individual):

        layers = individual.conv_layers + individual.dense_layers
        # select layer random 
        l = random.randint(0, len(layers)-1)
        layer = layers[l]

        if type(layer) is Layer:
            mutfunc = roulette([self.mutateSize, self.mutateDropout, self.mutateActivation,
                                self.randomInit],
                               [PROB_CHANGE_LAYER_SIZE, PROB_CHANGE_DROPOUT, PROB_CHANGE_ACTIVATION,
                               PROB_CHANGE_ALL])
            if mutfunc:
                mutfunc(layer)
        elif type(layer) is ConvLayer:
            mutfunc = roulette([self.mutateFilters, self.mutateKernelSize, self.mutateActivation,
                                self.randomInit],
                               [PROB_CHANGE_NUM_FILTERS, PROB_MUTATE_KERNEL_SIZE, PROB_CHANGE_CONV_ACTIVATION,
                                PROB_CHANGE_CONV_ALL])
            if mutfunc:
                mutfunc(layer)
        elif type(layer) is MaxPoolLayer:
            self.mutatePoolSize(layer)
        else:
            raise(TypeError("unknown type of layer"))
            
                
        return individual,

    def addLayer(self, individual):
        conv_part = random.randint(0,1)
        if (conv_part):
            l = random.randint(0, len(individual.conv_layers)-1)
            if random.randint(0,1):
                individual.conv_layers.insert(l, ConvLayer().randomInit())
            else:
                individual.conv_layers.insert(l, MaxPoolLayer().randomInit())
        else:
            l = random.randint(0, len(individual.dense_layers)-1)
            individual.dense_layers.insert(l, Layer().randomInit())
            
        return individual, 

    def delLayer(self, individual):
        conv_part = random.randint(0,1)
        if (conv_part):
            if len(individual.conv_layers)>1:
                l = random.randint(0, len(individual.conv_layers)-1)
                del individual.conv_layers[l]
        else:
            if len(individual.dense_layers)>1:
                l = random.randint(0, len(individual.dense_layers)-1)
                del individual.dense_layers[l]
                
        return individual,
    
    
    def mutate(self, individual): 
        
        mutfunc = roulette([self.mutateLayer, self.addLayer, self.delLayer],
                           [PROB_MUTATE_LAYER, PROB_ADD_LAYER, PROB_DEL_LAYER])
        if mutfunc:
            return mutfunc(individual)
        
        return individual, 


In [13]:
# Mutation ES

import random
import math


PROB_MUTATE_LAYER = 0.5
PROB_ADD_LAYER = 0.25
PROB_DEL_LAYER = 0.25

PROB_CHANGE_LAYER_SIZE = 0.4
PROB_CHANGE_DROPOUT = 0.3
PROB_CHANGE_ACTIVATION = 0.3

class MutationES:

    def __init__(self):
        pass


    def mutateSize(self, layer, strategy, size):
        # strategy coeficients 
        t = 1.0 / math.sqrt(2. * math.sqrt(size))
        t0 = 1.0 / math.sqrt(2. * size)
        n = random.gauss(0, 1)
        t0_n = t0 * n
        # modify strategy
        strategy.size *= math.exp(t0_n + t * random.gauss(0, 1))
        # modify size 
        layer.size += round(strategy.size * random.gauss(0, 1))
        layer.size = int(layer.size)
        if layer.size < Config.MIN_LAYER_SIZE:
            layer.size = Config.MIN_LAYER_SIZE
        if layer.size > Config.MAX_LAYER_SIZE:
            layer.size = Config.MAX_LAYER_SIZE

    def mutateDropout(self, layer, strategy, size):
        # strategy coeficients 
        t = 1.0 / math.sqrt(2. * math.sqrt(size))
        t0 = 1.0 / math.sqrt(2. * size)
        n = random.gauss(0, 1)
        t0_n = t0 * n
        # modify strategy
        strategy.dropout *= math.exp(t0_n + t * random.gauss(0, 1))
        # modify dropout 
        layer.dropout += strategy.dropout * random.gauss(0, 1)
        if layer.dropout < 0.0:
            layer.dropout = 0
        if layer.dropout > 0.9:
            layer.dropout = 0.9 
        
    def mutateActivation(self, layer, strategy, size):
        layer.activation = random.choice(Config.ACTIVATIONS)


    def mutateLayer(self, individual):

        # select layer random 
        l = random.randint(0, len(individual.layers)-1)
        layer = individual.layers[l]
        strategy = individual.strategy.blocks[l]
        
        mutfunc = roulette([self.mutateSize, self.mutateDropout, self.mutateActivation],
                           [PROB_CHANGE_LAYER_SIZE, PROB_CHANGE_DROPOUT, PROB_CHANGE_ACTIVATION])

        if mutfunc:
            mutfunc(layer, strategy, len(individual.layers))
        
        return individual

    def addLayer(self, individual):
        l = random.randint(0, len(individual.layers)-1)
        individual.layers.insert(l, Layer().randomInit())
        individual.strategy.blocks.insert(l, Block().randomInit())
        return individual 

    def delLayer(self, individual):
        if len(individual.layers)>1:
            l = random.randint(0, len(individual.layers)-1)
            del individual.layers[l]
            del individual.strategy.blocks[l]
        return individual, 
    
    def mutate(self, individual): 

        mutfunc = roulette([self.mutateLayer, self.addLayer, self.delLayer],
                           [PROB_MUTATE_LAYER, PROB_ADD_LAYER, PROB_DEL_LAYER])
        if mutfunc:
            return mutfunc(individual)

        assert len(individual.layers) == len(individual.strategy.blocks)
        
        return individual


In [14]:
# Pool, multiprocessing

from multiprocessing import Process, Queue

def worker(name, evalfunc, querries, answers):
    while True:
        querry = querries.get()
        #print("worker ", name, "evaluating")
        answer = evalfunc(querry)
        #print("worker ", name, "finished")
        answers.put(answer)

    
class Pool:
    """ Pool of workers. Workers consume tasks from the querries queue and 
        feed answers to the answers queue.
    """

    
    def __init__(self, processors, evalfunc):

        self.querries = Queue()
        self.answers = Queue()

        self.workers = []
        for i in range(processors):
             worker_i = Process(target=worker, args=(i, evalfunc, self.querries, self.answers))
             self.workers.append(worker_i)
             worker_i.start()        # Launch worker() as a separate python process

    def putQuerry(self, querry):
        self.querries.put(querry)
             
    def getAnswer(self):
        return self.answers.get()


    def close(self):
        for w in self.workers:
            w.terminate()
        #print("pool killed")
        


In [15]:
#Paralg

import random
import pickle
import time
import datetime 
from operator import attrgetter 

from deap import algorithms 

PROCESSORS = 10

def myAsyncEA(population, start_gen, toolbox, cxpb, mutpb, ngen,
              stats, halloffame, logbook, verbose, id=None):

    """ Asynchronous EA """ 

    def eval_individual(ind):
        ind.fitness.values = toolbox.evaluate(ind)
        return ind

    def gen_new_offspring():
        valid_offspring = True
        offspring = None
        while valid_offspring:
            # select two individuals and clone them 
            offspring = map(toolbox.clone, toolbox.select(population, 2))
            # apply crossover and mutation, take the first offspring
            offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)[0]
            valid_offspring = offspring.fitness.valid
            # if the offspring is valid it is already in the population
        return offspring 

    def log_stats():
        record = stats.compile(population)
        logbook.record(gen=start_gen + (num_evals // popsize),  **record)
        if verbose:
            print(logbook.stream)

    def save_checkpoint():
        cp = dict(population=population, generation=start_gen + (num_evals // popsize), halloffame=halloffame,
                  logbook=logbook, rndstate=random.getstate())
        if id is None:
            cp_name = "checkpoint_ea.pkl"
        else:
            cp_name = "checkpoint_ea_{}.pkl".format(id)
        pickle.dump(cp, open(cp_name, "wb"))
            
    
    popsize = len(population)
    MIN_POP_SIZE = popsize // 2 
    total_time = datetime.timedelta(seconds=0)

    pool = Pool(processors=PROCESSORS, evalfunc=eval_individual)    

    db = Database()
    
    if all([ind.fitness.valid for ind in population]):
        print("All individuals have valid fitness")
        # all inds are valid (loaded from checkpoint), generate some new to fill the pool 
        for _ in range(PROCESSORS):
            offspring = gen_new_offspring() 
            pool.putQuerry(offspring)
    else:
        # put all individuals to the pool 
        for ind in population:
            pool.putQuerry(ind)
        population = [] # we get them back later 

    num_evals = 0

    start_time = datetime.datetime.now()
    while num_evals < (ngen-start_gen)*popsize:

        # get finished individual and add him to population 
        ind = pool.getAnswer()
        assert ind.fitness.valid
        # add fitness and individual to the database
        db.insert(ind, ind.fitness.values[0])
        population.append(ind)
        num_evals += 1
        
        # update halloffame
        halloffame.update([ind])

        if len(population) < MIN_POP_SIZE:
            continue

        if num_evals % popsize == 0:
            # record statistics and save checkpoint
            log_stats()            
            save_checkpoint()
            
        # check time    
        eval_time = datetime.datetime.now() - start_time
        total_time = total_time + eval_time
        #print("Time ", total_time)
        if total_time > datetime.timedelta(hours=8*24):
            print("Time limit exceeded.")
            break
        start_time = datetime.datetime.now() 

        
        if len(population) > popsize:
            # delete the worst one
            population = list(sorted(population, key=attrgetter('fitness')))[1:]

        # generate new offspring and put him to the pool 
        offspring = gen_new_offspring() 
        pool.putQuerry(offspring)


    pool.close()
    #    print(db.data)
    db.save("database." + id + ".pkl")
    return population, logbook


In [16]:
# Strategy 

import random

class Block:

    """ Strategy for one layer.
    """ 
    def __init__(self):
        pass

    def randomInit(self):
        self.size = random.uniform(Config.SIZE_MIN_STRATEGY, Config.SIZE_MAX_STRATEGY)
        self.dropout = random.uniform(Config.DROPOUT_MIN_STRATEGY, Config.DROPOUT_MAX_STRATEGY)
        return self


class Strategy:

    def randomInit(self, size):
        self.blocks = [ Block().randomInit() for _ in range(size) ]

        
def initIndStrategy(indclass, strategyclass):
    ind = indclass()
    ind.randomInit()
    ind.strategy = strategyclass()
    ind.strategy.randomInit(len(ind.layers))
    return ind 


In [18]:
# MAIN

import sys
import array
import random
import pickle
import numpy as np
import multiprocessing 


from deap import algorithms
from deap import base
from deap import creator
from deap import tools


#import argparse

#parser = argparse.ArgumentParser()
#parser.add_argument('--trainset', help='filename of training set')
#parser.add_argument('--testset', help='filename of test set')
#parser.add_argument('--id', help='computation id')
#parser.add_argument('--checkpoint', help='checkpoint file to load the initial state from')
#parser.add_argument('--config', help='json config filename')

#args = parser.parse_args()
##trainset_name = args.trainset
#testset_name = args.testset 
#id = args.id
#if id is None:
    ##id = "" 
#checkpoint_file = args.checkpoint
#config_name = args.config
#if config_name is not None:
    #load_config(config_name)

# for classification fitness is accuracy, for approximation fitness is error
if Config.task_type == "classification":
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
else:
    creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", Individual, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Structure initializers
toolbox.register("individual", initIndividual, creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# use multiple processors 
pool = multiprocessing.Pool(5)
toolbox.register("map", pool.map)


# load the whole data 
X_train, y_train, X_test, y_test = load_dataset()

# register operators 
fit = Fitness(X_train, y_train)
mut = Mutation()
cross = Crossover()

toolbox.register("evaluate", fit.evaluate)
toolbox.register("mate", cross.cxOnePoint)
toolbox.register("mutate", mut.mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

def main(id, checkpoint_name=None):
    # random.seed(64)

    if checkpoint_name:
        # A file name has been given, then load the data from the file
        cp = pickle.load(open(checkpoint_name, "rb"))
        pop = cp["population"]
        start_gen = cp["generation"] + 1
        hof = cp["halloffame"]
        logbook = cp["logbook"]
        random.setstate(cp["rndstate"])
    else:
        pop = toolbox.population(n=Config.pop_size)
        start_gen = 0
        hof = tools.HallOfFame(1)
        logbook = tools.Logbook()
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, log = myEASimple(pop, start_gen, toolbox, cxpb=0.6, mutpb=0.2, ngen=Config.ngen, 
                              stats=stats, halloffame=hof, logbook=logbook, verbose=True,
                              id=id)

    return pop, log, hof



# load the whole data 
#X_train, y_train, X_test, y_test = load_dataset()

    
# set cfg
Config.input_shape = X_train[0].shape 
Config.noutputs = y_train.shape[1]
    #    print(Config.input_shape, Config.noutputs)



#if checkpoint_file is None:
pop, log, hof = main(id)
#else:
    #pop, log, hof = main(id, checkpoint_file)

    
network = hof[0].createNetwork()
network.summary()
print( hof[0] )
print( hof[0].fitness )


    # learn on the whole set
    #    
E_train, E_test = [], []  

for _ in range(10):
  network = hof[0].createNetwork() 
  network.fit(X_train, y_train, batch_size=Config.batch_size, nb_epoch=Config.epochs, verbose=0)

  yy_train = network.predict(X_train)
  E_train.append(error(yy_train, y_train)) 
        
  yy_test = network.predict(X_test)
  E_test.append(error(yy_test, y_test))

        
def print_stat(E, name):
    print("E_{:6} avg={:.4f} std={:.4f}  min={:.4f} max={:.4f}".format(name,
                                                                           np.mean(E),
                                                                           np.std(E),
                                                                           np.min(E),
                                                                           np.max(E)))
        
print_stat(E_train, "train")
print_stat(E_test, "test")

    

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Data loaded
Preparing pixels
Pixel data ready


ValueError: ignored